In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
# Load your dataset
file_path = ""  # Update this path to your dataset file
df = pd.read_csv(file_path)

In [99]:
df1=df[df['SubstanceUse']==1]

In [112]:
df.shape

(38950, 9)

In [111]:
df.head(3)

,Unnamed: 0.1,Unnamed: 0,author,created_utc,selftext,title,subreddit,date,posts
0,97258,300155,Hajimemeforme,2018-08-18 09:41:01,there is a day to celebrate love in our countr...,TIFU,teenagers,2018-08-18 09:41:01,tifu day celebrate love country 7/7 lunar cale...
1,79310,244446,Slayyr_,2018-12-11 04:15:04,"\n\n\n\nliterally say anything to anyone, 80% ...",How to make friends in high school,teenagers,2018-12-11 04:15:04,make friends high school literally say anythin...
2,46661,147104,ScndChoice,2018-10-06 19:38:11,i accidentally trew away all my notes for a ph...,Am I retarded or just really dumb,teenagers,2018-10-06 19:38:11,retarded really dumb accidentally trew away no...


In [3]:
# Define emotions based on ISEAR
all_emotions = ['joy', 'guilt', 'anger', 'disgust', 'fear', 'sadness', 'shame']

In [4]:
# System prompt for GPT-4
SYSTEM_PROMPT_CLASSIFY_ISEAR = ' '.join([
    "You are an emotionally-intelligent and empathetic agent.",
    "You will be given a piece of text, and your task is to identify all the emotions expressed by the writer of the text.",
    "You are only allowed to make one selection from the following emotions, and don't use any other words: {}"
]).format(', '.join(all_emotions))


In [ ]:
# Function to get emotion classification from GPT-4
def classify_emotion(text):
    try:   
    # Call to LLM (example using OpenAI API)
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_CLASSIFY_ISEAR},
                {"role": "user", "content": text}
                    ],
        model="gpt-4o-mini",
        #model="gpt-3.5",
        #max_tokens=4096,
        temperature=0.5
            )
        # Extract the response text
        emotion_label = response.choices[0].message.content.strip().lower()
        if emotion_label in all_emotions:
            return emotion_label
        else:
            return 'neutral'  # Return None if the label doesn't match any known emotion

    except Exception as e:
        print(f"Error processing text: {e}")
        return None

In [113]:
classify_emotion('i am happy ')

'joy'

In [114]:
# Apply GPT-4 emotion classification to each post
tqdm.pandas()
df['emotion']= df['posts'].progress_apply(classify_emotion)

100%|██████████| 38950/38950 [4:40:17<00:00,  2.32it/s]    


In [116]:
df.head(3)

,Unnamed: 0.1,Unnamed: 0,author,created_utc,selftext,title,subreddit,date,posts,emotion
0,97258,300155,Hajimemeforme,2018-08-18 09:41:01,there is a day to celebrate love in our countr...,TIFU,teenagers,2018-08-18 09:41:01,tifu day celebrate love country 7/7 lunar cale...,sadness
1,79310,244446,Slayyr_,2018-12-11 04:15:04,"\n\n\n\nliterally say anything to anyone, 80% ...",How to make friends in high school,teenagers,2018-12-11 04:15:04,make friends high school literally say anythin...,fear
2,46661,147104,ScndChoice,2018-10-06 19:38:11,i accidentally trew away all my notes for a ph...,Am I retarded or just really dumb,teenagers,2018-10-06 19:38:11,retarded really dumb accidentally trew away no...,anger


In [ ]:
df.to_csv('',index=False)